In [1]:
import os
#os.chdir('pytrends/')

import sys
sys.path.insert(0,os.getcwd())

print(os.getcwd())
print(sys.path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from typing import List, Dict

from pytrends.request import TrendReq

from tqdm import tqdm
from itertools import islice
from IPython.display import display
import time

from warnings import warn
import json

/Users/annaster/Documents/TechLabs/NaturDoc/anna_code/notebooks
['/Users/annaster/Documents/TechLabs/NaturDoc/anna_code/notebooks', '/Users/annaster/Documents/TechLabs/NaturDoc/anna_code/notebooks', '/Users/annaster/opt/anaconda3/lib/python39.zip', '/Users/annaster/opt/anaconda3/lib/python3.9', '/Users/annaster/opt/anaconda3/lib/python3.9/lib-dynload', '', '/Users/annaster/opt/anaconda3/lib/python3.9/site-packages', '/Users/annaster/opt/anaconda3/lib/python3.9/site-packages/aeosa', '/Users/annaster/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/Users/annaster/.ipython']


In [2]:
def pytrend_score_group(terms : list, print_table : bool = False) -> Dict[str,int]:
    pt = TrendReq(retries=5)
    pt.build_payload(terms)
    tmp_df = pt.interest_over_time()
    tmp_df = tmp_df[terms].mean()
    
    if print_table:
        display(tmp_df)
        
    ranks = tmp_df.rank().to_dict()
    return ranks 


def break_into_groups(lst, group_size):
    """
    Breaks a list into groups of a specified size.
    
    Args:
        lst (list): The list to be broken into groups.
        group_size (int): The size of each group.
    
    Returns:
        list: A list of lists, where each inner list contains `group_size`
        elements from the original list.
    """
    return [lst[i:i+group_size] for i in range(0, len(lst), group_size)]


In [3]:
# Load the JSON file
with open('remedies.json', 'r') as f:
    remedies = json.load(f)

In [4]:
remedies[:1]

[{'TAXON': 'Abelmoschus esculentus',
  'ACTIVITY': 'Abortifacient,Antidote,Boil,Burn,Catarrh,Coffee,Colic,Cordial,Demulcent,Diarrhea,Diuretic,Dyspepsia,Dysuria,Emollient,Gonorrhea,Inflammation,Intestine,Lactogogue,Leprosy,Parturition,Pectoral,Scald,Sore,Spasm,Stimulant,Sudorific,Syphilis,Ache(Tooth),Tumor,Whitlow,Abscess,Aphrodisiac,Gum,Labor,Pregnancy,Wound',
  'CNAME': "Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,Gombo,Gumbo,Calalou,Calalou Gombo,Quingombo,Ochro",
  'FAMILY': 'Malvaceae',
  'GENUS': 'Abelmoschus',
  'SPECIES': 'esculentus',
  'VERNAC': None,
  'CLINICAL': None,
  'TRADITIONAL': None,
  'FOLK': None,
  'CONTRAINDICATION': None,
  'WARNING': None,
  'ADVERSE': None,
  'POSOLOGY': None}]

In [5]:
# Filter remedies by VERNAC
enriched_remedies = [remedy for remedy in remedies if  remedy['VERNAC']]

# Print the number of enriched remedies
print(f"Number of enriched remedies: {len(enriched_remedies)}")

Number of enriched remedies: 75


In [23]:
 #Filter enriched_remedies by CNAME
filtered_remedies = [remedy for remedy in enriched_remedies if remedy.get('CNAME')]
print(f"Number of filtered remedies: {len(filtered_remedies)}")

Number of filtered remedies: 66


In [24]:
remedies = list({r['TAXON']: r for r in filtered_remedies}.values())
print(f"Number of unique remedies: {len(remedies)}")

Number of unique remedies: 66


In [26]:
remedies_names = [remedy['TAXON'][0] if isinstance(remedy['TAXON'], list) else remedy['TAXON'] for remedy in remedies if remedy.get('TAXON')]
print(remedies_names)

['Aesculus hippocastanum', 'Allium cepa', 'Allium sativum', 'Aloe africana', 'Aloe ferox', 'Aloe sp', 'Aloe spicata', 'Aloe vera', 'Althaea officinalis', 'Andrographis paniculata', 'Angelica sinensis', 'Arctostaphylos uva-ursi', 'Astragalus membranaceus', 'Astragalus mongholicus', 'Brucea javanica', 'Bupleurum falcatum', 'Calendula officinalis', 'Centella asiatica', 'Chrysanthemum parthenium', 'Cimicifuga racemosa', 'Cinnamomum cassia', 'Cinnamomum verum', 'Coptis chinensis', 'Coptis japonica', 'Crataegus monogyna', 'Crataegus sp', 'Curcuma longa', 'Echinacea angustifolia', 'Echinacea pallida', 'Ephedra sinica', 'Eucalyptus globulus', 'Eucalyptus sp', 'Ginkgo biloba', 'Glycyrrhiza glabra', 'Glycyrrhiza uralensis', 'Hamamelis virginiana', 'Hypericum perforatum', 'Matricaria chamomilla', 'Melissa officinalis', 'Mentha piperita', 'Ocimum sanctum', 'Oenothera biennis', 'Paeonia lactiflora', 'Panax ginseng', 'Piper methysticum', 'Plantago asiatica', 'Plantago ovata', 'Platycodon grandifloru

In [27]:
# Break remedies into groups of 5
groups = break_into_groups(remedies_names, 5)

In [28]:
groups

[['Aesculus hippocastanum',
  'Allium cepa',
  'Allium sativum',
  'Aloe africana',
  'Aloe ferox'],
 ['Aloe sp',
  'Aloe spicata',
  'Aloe vera',
  'Althaea officinalis',
  'Andrographis paniculata'],
 ['Angelica sinensis',
  'Arctostaphylos uva-ursi',
  'Astragalus membranaceus',
  'Astragalus mongholicus',
  'Brucea javanica'],
 ['Bupleurum falcatum',
  'Calendula officinalis',
  'Centella asiatica',
  'Chrysanthemum parthenium',
  'Cimicifuga racemosa'],
 ['Cinnamomum cassia',
  'Cinnamomum verum',
  'Coptis chinensis',
  'Coptis japonica',
  'Crataegus monogyna'],
 ['Crataegus sp',
  'Curcuma longa',
  'Echinacea angustifolia',
  'Echinacea pallida',
  'Ephedra sinica'],
 ['Eucalyptus globulus',
  'Eucalyptus sp',
  'Ginkgo biloba',
  'Glycyrrhiza glabra',
  'Glycyrrhiza uralensis'],
 ['Hamamelis virginiana',
  'Hypericum perforatum',
  'Matricaria chamomilla',
  'Melissa officinalis',
  'Mentha piperita'],
 ['Ocimum sanctum',
  'Oenothera biennis',
  'Paeonia lactiflora',
  'Pana